In [ ]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random
import re
import unicodedata
import sys

In [ ]:
# restore all of our data structures
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']
classes_details = data['classes_details']

In [ ]:
import urllib.request
import urllib.response
import sys
import os, glob
import http.client, urllib
import json
import http.client, urllib.request, urllib.parse, urllib.error, base64

# Replace the accessKey string value with your valid access key.
accessKey = '82e4612615634c398bfd26e7d6327833'
url = 'westcentralus.api.cognitive.microsoft.com'
path = '/text/analytics/v2.0/keyPhrases'

def extract_keywords(body):
    headers = {'Ocp-Apim-Subscription-Key': accessKey}
    conn = http.client.HTTPSConnection(url)
    body_req = {'documents':[{'language': 'en', 'id': 1, 'text': body}]}
    body_json = json.dumps(body_req)
    conn.request ("POST", path, body_json, headers)
    response = conn.getresponse ()
    string = response.read().decode('utf-8')
    json_obj = json.loads(string)
    print("json_obj: ", json_obj)
    keyphrases_list = ((json_obj['documents'])[0])['keyPhrases']
    print("keyphrases_list: ", keyphrases_list)
    return keyphrases_list

In [ ]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

In [ ]:
## a table structure to hold the different punctuation used

## method to remove punctuations from sentences.
# def remove_punctuation(text):
#     return text.translate(tbl)

def clean_up_sentence(sentence):
    #strip punctuations, numbers, and words containing numbers
    no_nums1 = ' '.join(s for s in sentence.split() if not any(c.isdigit() for c in s))
    no_nums2 = re.sub(r'\d+', '', no_nums1)
    keywords = extract_keywords(no_nums2)
    keyword_str = ' '.join(s for s in keywords)
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(keyword_str)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    print("sentence after stemming: ", sentence_words)
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if s in w.strip().split(): 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))